In [1]:
LLAMA = "llama2-70b-4096"
GEMMA = "gemma-7b-it"
MIXTRAL = "mixtral-8x7b-32768"

oracion = "El susurro del viento me transportó a un estado de serenidad profunda que no había experimentado antes."
palabra = "profunda"

In [184]:
from groq import Groq
import json

def get_groq_simplifications(model: str, oracion: str, palabra: str):
    client = Groq(
        api_key="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",
    )

    oracion_ejemplo = "El resplandeciente sol iluminaba el paisaje selvático, creando un efecto casi mágico en la vegetación exuberante que se extendía a lo lejos."
    palabra_ejemplo = "mágico"

    params = {
        "messages": [
            {
                "role": "system",
                "content": f"""
                    Tarea: Sustituir una palabra en una oración sin cambiar el orden de las demás palabras.
                    Respuesta: La respuesta solo incluye los candidatos. Tu respuesta no debe incluir todas las oraciones, solo las palabras por las que se quiere sustituir. La respuesta debe ser en el siguiente formato JSON:
                        {{
                            "candidatos": ["candidato1", "candidato2", "candidato3", "candidato4", "candidato5"],
                        }}
                """
            },
            {
                "role": "user",
                "content": f"""
                    La oración original es: "{oracion}" La palabra a sustituir es "{palabra}".
                    Instrucciones: Proporciona cinco candidatos para sustituir la palabra "{palabra}" en la oración, manteniendo el significado general y la coherencia de la oración. Los candidatos no deben alterar el orden de las demás palabras en la oración.
                """
            }
        ],
        "model": f"{model}",  # Aquí debes reemplazar "model" por el identificador de modelo real que deseas utilizar.
        "max_tokens": 512,
        "temperature": 0.1,
    }


    if model != LLAMA: params["response_format"] = { "type": "json_object" }

    chat_completion = client.chat.completions.create(**params)

    output = json.loads(chat_completion.choices[0].message.content)
    output['candidatos'].append(palabra)

    return output

#get_groq_simplifications(GEMMA, oracion, palabra)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/CREA_total.TXT', sep='\t', header=None,
                 names=['Orden', 'Palabra', 'Frec.absoluta', 'Frec.normalizada'],
                 encoding='ISO 8859-1', low_memory=False,
                 dtype={'Orden': str, 'Palabra': str, 'Frec.absoluta': str, 'Frec.normalizada': float})


# Asegurarse de que 'Frec.normalizada' se interprete como numérico
df['Frec.normalizada'] = pd.to_numeric(df['Frec.normalizada'], errors='coerce')

# Ordenar el DataFrame por 'Frec.normalizada' para un mejor gráfico
df_sorted = df.sort_values(by='Frec.normalizada', ascending=True)

# Suponiendo que ya has cargado los datos en el DataFrame df
# Convertir explícitamente la columna 'Palabra' a cadenas de texto
df['Palabra'] = df['Palabra'].astype(str)

# Ordenar el DataFrame por 'Frec.normalizada' para un mejor gráfico
df_sorted = df.sort_values(by='Frec.normalizada', ascending=True)

# Convertir 'Frec.normalizada' a numérico, por si acaso no lo está
df['Frec.normalizada'] = pd.to_numeric(df['Frec.normalizada'], errors='coerce')

In [5]:
df.head()

,Orden,Palabra,Frec.absoluta,Frec.normalizada
0,Orden,Frec.absoluta,Frec.normalizada,NaN
1,1.,de,"9,999,518",65545.55
2,2.,la,"6,277,560",41148.59
3,3.,que,"4,681,839",30688.85
4,4.,el,"4,569,652",29953.48


In [6]:
import pandas as pd
import unicodedata
import json

# Función para remover acentos y caracteres diacríticos
def remover_acentos(input_str):
    if isinstance(input_str, float):  # Verifica si el valor es un float (lo que usualmente indica un NaN en pandas)
        return ""  # Devuelve una cadena vacía si el valor es NaN
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join(c for c in nfkd_form if not unicodedata.combining(c))

def obtener_frecuencias_de_sustitutos(candidatos):
    # Leer el DataFrame asegurándose de que las columnas tienen el tipo correcto
    # Es posible que necesites ajustar los tipos dependiendo de los datos
    df = pd.read_csv('data/CREA_total.TXT', sep='\t', header=None,
                    names=['Orden', 'Palabra', 'Frec.absoluta', 'Frec.normalizada'],
                    encoding='ISO 8859-1', low_memory=False,
                    dtype={'Orden': str, 'Palabra': str, 'Frec.absoluta': str, 'Frec.normalizada': float})

    # Crear una nueva lista con las palabras sin acentos y en minúsculas
    candidatos_minuscula = [remover_acentos(palabra).lower() for palabra in candidatos["candidatos"]]

    # Normalizar la columna 'Palabra' en el DataFrame para que coincida con la lista de candidatos
    df['Palabra'] = df['Palabra'].astype(str).apply(remover_acentos).str.lower().str.strip()

    # Filtrar el DataFrame para obtener solo las filas que contienen las palabras candidatas
    df_filtrado = df[df['Palabra'].isin(candidatos_minuscula)]

    df_unicas_filtradas = df_filtrado.drop_duplicates(subset='Palabra')

    return df_unicas_filtradas

In [249]:
def obtener_predicciones(oracion_p: str, palabra_p: str):
    candidatos_mixtral = get_groq_simplifications(MIXTRAL, oracion_p, palabra_p)
    df_filtrado_mixtral = obtener_frecuencias_de_sustitutos(candidatos_mixtral)

    candidatos_llama = get_groq_simplifications(LLAMA, oracion_p, palabra_p)
    df_filtrado_llama = obtener_frecuencias_de_sustitutos(candidatos_llama)

    candidatos_gemma = get_groq_simplifications(GEMMA, oracion_p, palabra_p)
    df_filtrado_gemma = obtener_frecuencias_de_sustitutos(candidatos_gemma)

    # combinamos filtrados de los modelos
    df_merged = pd.concat([df_filtrado_mixtral, df_filtrado_llama, df_filtrado_gemma], ignore_index=True)

    # Ordenamos segun frecuencia absoluta
    df_merged = df_merged.sort_values('Frec.absoluta', ascending=False)

    # Elimino duplicados
    df_merged = df_merged.drop_duplicates(subset='Palabra')

    return df_merged[:10]


In [250]:
# Delete this if necesary, LLAMA2 is works badly with JSON response output

def obtener_predicciones_v2(oracion_p: str, palabra_p: str):
    candidatos_mixtral = get_groq_simplifications(MIXTRAL, oracion_p, palabra_p)
    df_filtrado_mixtral = obtener_frecuencias_de_sustitutos(candidatos_mixtral)

    candidatos_gemma = get_groq_simplifications(GEMMA, oracion_p, palabra_p)
    df_filtrado_gemma = obtener_frecuencias_de_sustitutos(candidatos_gemma)

    # combinamos filtrados de los modelos
    df_merged = pd.concat([df_filtrado_mixtral, df_filtrado_gemma], ignore_index=True)

    # Ordenamos segun frecuencia absoluta
    df_merged = df_merged.sort_values('Frec.absoluta', ascending=False)

    # Elimino duplicados
    df_merged = df_merged.drop_duplicates(subset='Palabra')

    return df_merged[:10]

In [251]:

def obtener_predicciones_v3(oracion_p: str, palabra_p: str):
    candidatos_mixtral = get_groq_simplifications(MIXTRAL, oracion_p, palabra_p)
    df_merged = obtener_frecuencias_de_sustitutos(candidatos_mixtral)

    # Ordenamos segun frecuencia absoluta
    df_merged = df_merged.sort_values('Frec.absoluta', ascending=False)

    # Elimino duplicados
    df_merged = df_merged.drop_duplicates(subset='Palabra')

    return df_merged[:10]

In [253]:
# Trial data

spanish_input_file = "/home/spoturno/coding/MLSP_Data/Data/Test/Spanish/multilex_test_es_ls_unlabelled.tsv" # change this
output_file = "/home/spoturno/coding/MLSP/predictions/groq_models_with_crea_corpus_v2.tsv" # change this

sentences = []
target_words = []

cant_ejemplos = 593
i = 0

with open(spanish_input_file, 'r', encoding='utf-8') as file, open(output_file, 'w', encoding='utf-8') as out_file:
    for line in file:
        i += 1

        # Separamos la linea en tabulaciones 
        parts = line.strip().split('\t')
        sentence = parts[0]
        target_word = parts[1]

        # Guardo la oracion y palabra ordenadamente
        sentences.append(sentence)
        target_words.append(target_word)

        try:
            predicciones = obtener_predicciones(sentence, target_word)
            predictions = predicciones['Palabra'].to_list()
            output_line = f"{sentence}\t{target_word}\t" + '\t'.join(predictions) + '\n'
            out_file.write(output_line)
            print(f"Linea {i}/{cant_ejemplos} \u2713")
            continue
        except Exception as e:
            None

        try:
            predicciones = obtener_predicciones_v2(sentence, target_word)
            predictions = predicciones['Palabra'].to_list()
            output_line = f"{sentence}\t{target_word}\t" + '\t'.join(predictions) + '\n' 
            out_file.write(output_line)
            print(f"Linea {i}/{cant_ejemplos} \u2713")
            continue
        except Exception as e:
            None
            
        try:
            predicciones = obtener_predicciones_v3(sentence, target_word)
            predictions = predicciones['Palabra'].to_list()
            output_line = f"{sentence}\t{target_word}\t" + '\t'.join(predictions) + '\n' 
            out_file.write(output_line)
            print(f"Linea {i}/{cant_ejemplos} \u2713")
            continue
        except Exception as e:
            print(f"Error procesando linea {i}: {e}")

        output_line = "ERRORLINE\n"
        out_file.write(output_line)

        print(f"Linea {i}/{cant_ejemplos} \u2713")

    if i == cant_ejemplos: print("Completado!")

Linea 1/593 ✓
Linea 2/593 ✓
Linea 3/593 ✓
Linea 4/593 ✓
Linea 5/593 ✓
Linea 6/593 ✓
Linea 7/593 ✓
Linea 8/593 ✓
Linea 9/593 ✓
Linea 10/593 ✓
Linea 11/593 ✓
Linea 12/593 ✓
Linea 13/593 ✓
Linea 14/593 ✓
Linea 15/593 ✓
Linea 16/593 ✓
Linea 17/593 ✓
Linea 18/593 ✓
Linea 19/593 ✓
Linea 20/593 ✓
Linea 21/593 ✓
Linea 22/593 ✓


KeyboardInterrupt: 